### Elementy sztucznej inteligencji
# Projekt: Binarne Drzewa Decyzyjne
autorzy:
1. Arkadiusz Florek
2. Maciej Komosa
3. Albert Pieniądz
4. Jakub Zięba

In [1]:
import numpy as np, pandas as pd, math as mat

Funkcje do obliczania entropii dla danej konkluzji

In [27]:
def prawdopodobienstwo(wartosci, unikatowe_wartosci):

    # tabela zawierająca częstości wystąpienia poszczególnych wartości przesłanek
    p_tab = np.zeros(len(unikatowe_wartosci))

    for indeks, unikatowa_wartosc in enumerate(unikatowe_wartosci):

        for wartosc in wartosci:

            if wartosc == unikatowa_wartosc:
                p_tab[indeks] += 1

    p_tab = p_tab / len(wartosci)
    return p_tab


In [3]:
def entropia(p_tab):
    I = 0
    for p in p_tab:
        if p == 0:
            continue
        else:
            I += -p*mat.log2(p)
    return I

Klasa węzła

In [4]:
class wezel:

    def __init__(self):
        self.value = None
        self.next = None
        self.childs = None

Importowanie danych

In [5]:
data = pd.read_csv("sport.csv")
data.head()
X = data.iloc[:, 0:-1]
Y = data.iloc[:, -1]

In [35]:
p_tab = []; p_buf = []
e_tab = []; e_buf = []

for przeslanka in range(X.shape[1]):

    for konkluzja in range(len(set(Y))):

        wartosci = X.iloc[:, przeslanka]
        
        konkluzja += 1
        unikatowe_wartosci = list(set(wartosci))

        wartosci = wartosci[Y == konkluzja]
    
        p_buf.append(prawdopodobienstwo(wartosci, unikatowe_wartosci))
        e_buf.append(entropia(p_buf[-1]))

    p_tab.append(p_buf)
    e_tab.append(e_buf)
    p_buf = []
    e_buf = []

In [33]:
# entropia inicjalna
entropia(prawdopodobienstwo(Y, set(Y)))

[[0.9252253278054432, 0.7950402793845223, 0.9509560484549725, 0.6840384356390417, 0.7219280948873623, 0.8865408928220899, 0.9456603046006402], [1.5441141771508817, 1.5690255736436507, 1.5448491887169091, 1.5300303808171334, 1.5413152774012366, 1.574331785544523, 0.9940302114769565], [1.4870439806899802, 1.508894704985911, 1.5610150105933092, 1.580982641705794, 1.404107451387086, 1.4224751175914554, 1.3221793455166666], [1.5450122867835447, 1.5609563153489607, 1.3800943362833298, 1.4949188482339508, 1.55352095112747, 1.4777468341000444, 1.5726236638951638], [1.9330643714792726, 1.9408786445478214, 1.584962500721156, 1.893666744453497, 1.4986885704510973, 1.7349112826910291, 1.3221793455166666]]


In [36]:
I_tab = np.zeros(len(set(Y)))
I_buf = 0

for indeks, I in enumerate(I_tab):
    
    for e in e_tab:
        I_buf += e[indeks]
    
    I = I_buf
    I_buf = 0

I_tab

array([0., 0., 0., 0., 0., 0., 0.])